# Blogs related to this exercise:

[Average Face : OpenCV ( C++ / Python ) Tutorial](https://www.learnopencv.com/average-face-opencv-c-python-tutorial/)

In [1]:
import cv2
import dlib
import numpy as np
import math

from glob import glob

In [2]:
# Get face landmarks

def get_landmarks(filename, predictor_path = 'data/shape_predictor_68_face_landmarks.dat'):
    
    # Instantiate a face detector
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    
    img = dlib.load_rgb_image(filename)

    # Ask the detector to find the bounding boxes of each face. The 1 in the
    # second argument indicates that we should upsample the image 1 time. This
    # will make everything bigger and allow us to detect more faces.
    detected = detector(img, 1) # Return corresponding rectangles for the detected faces
    for index, box in enumerate(detected):
        landmarks = predictor(img, box)  
        coords = []
        for i in range(0, 68):
            coords.append((landmarks.part(i).x, landmarks.part(i).y))

    return coords

In [3]:
# Computing similarity transform given two sets of reference points
# OpenCV requires 3 pairs of corresponding points
# We are faking the third one by forming a equilateral 
# triangle with the existing two points

def similarity_transform(src_points, dst_points):
    s60 = math.sin(60 * math.pi / 180)
    c60 = math.cos(60 * math.pi / 180)
    
    src_points = np.copy(src_points).tolist()
    dst_points = np.copy(dst_points).tolist()
    
    # Faked source reference point 
    src_x = c60 * (src_points[0][0] - src_points[1][0]) - s60 * (src_points[0][1] - src_points[1][1]) + src_points[1][0]
    src_y = s60 * (src_points[0][0] - src_points[1][0]) + c60 * (src_points[0][1] - src_points[1][1]) + src_points[1][1]
    
    src_points.append([np.int(src_x), np.int(src_y)])
    
    # Faked destination reference point
    dst_x = c60 * (dst_points[0][0] - dst_points[1][0]) - s60 * (dst_points[0][1] - dst_points[1][1]) + dst_points[1][0]
    dst_y = s60 * (dst_points[0][0] - dst_points[1][0]) + c60 * (dst_points[0][1] - dst_points[1][1]) + dst_points[1][1]
    
    dst_points.append([np.int(dst_x), np.int(dst_y)])
    
    # Get the corresponding rigid transform matrix
    transform_mat = cv2.estimateRigidTransform(np.array(src_points), np.array(dst_points), False)
    
    return transform_mat

In [4]:
# Check if a point is inside of a rectangle, this is a must for `Subdiv2D`
def is_inside(rect, point):
    if point[0] < rect[0]:
        return False
    if point[1] < rect[1]:
        return False
    if point[0] > rect[2]:
        return False
    if point[1] > rect[3]:
        return False
    return True

# Calculate Delaunay triangles
def calculate_delaunay_triangles(rect, points):
    # Create subdivisions
    subdiv = cv2.Subdiv2D(rect)
    
    # Insert points into subdiv
    for p in points:
        subdiv.insert((p[0], p[1]))
    
    # List of possible triangles can be formed by above points
    triangles = subdiv.getTriangleList()
    print("Totally there are {} triangles generated".format(len(triangles)))
    
    # Find the vertex indicies of triangles in the points array
    delaunay_triangles = []
    
    for t in triangles:
        pts = []
        pts.append((t[0], t[1]))
        pts.append((t[2], t[3]))
        pts.append((t[4], t[5]))
        
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])
        
        if is_inside(rect, pt1) and is_inside(rect, pt2) and is_inside(rect, pt3):
            indicies = []
            for i in range(0, 3):
                for j in range(0, len(points)):
                    if abs(pts[i][0] - points[j][0]) < 1.0 and abs(pts[i][1] - points[j][1]) < 1.0:
                        indicies.append(j)
            if len(indicies) == 3:
                delaunay_triangles.append((indicies[0], indicies[1], indicies[2]))
    print("Totally there are {} Delaunay triangles".format(len(delaunay_triangles)))
    
    return delaunay_triangles

In [11]:
# Apply affine transform calculated using src_ri and dst_ri to src and output an image of certain size
def apply_affine_transform(src, src_tri, dst_tri, size):
    # Given a pair of triangles, find the affine transform needed
    warp_mat = cv2.getAffineTransform(np.float32(src_tri), np.float32(dst_tri))
    
    # Apply the above Affine transform to the src image
    dst = cv2.warpAffine(src, warp_mat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)
    
    return dst
    

def warp_triangle(img1, img2, t1, t2):
    # In openCV, affine is applied to rectangle images but not triangle regions
    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    
    # Offset points by left top corner of the respective rectangles
    t1_rect = []
    t2_rect = []
    t2_rect_int = []
    
    for i in range(0, 3):
        t1_rect.append(((t1[i][0] - r1[0]), (t1[i][1] - r1[1])))
        t2_rect.append(((t2[i][0] - r2[0]), (t2[i][1] - r2[1])))
        t2_rect_int.append(((t2[i][0] - r2[0]), (t2[i][0] - r2[1])))
        
    # Get a mask by filling the triangle
    mask = np.zeros((r2[3], r2[2], 3), dtype=np.float32)
    cv2.fillConvexPoly(mask, np.int32(t2_rect_int), (1.0, 1.0, 1.0), 16, 0)
    
    # Apply warp to a small rectanglar area of the image
    img1_rect = img1[r1[1]: r1[1]+r1[3], r1[0]: r1[0]+r1[2]]
    
    # Desired image size for the rectanglar area
    size = (r2[2], r2[3])
    
    img2_rect = apply_affine_transform(img1_rect, t1_rect, t2_rect, size)
    img2_rect = img2_rect * mask
    
    img2[r2[1]: r2[1]+r2[3], r2[0]: r2[0]+r2[2]] = img2[r2[1]: r2[1]+r2[3], r2[0]: r2[0]+r2[2]] * ((1.0, 1.0, 1.0) - mask)
    img2[r2[1]: r2[1]+r2[3], r2[0]: r2[0]+r2[2]] = img2[r2[1]: r2[1]+r2[3], r2[0]: r2[0]+r2[2]] + img2_rect


In [5]:
img_path = 'images/presidents'

facial_images = []
facial_landmarks = []

for file in glob('images/presidents/*'):
    if file.endswith('.jpg'):
        # Read in images one by one and convert them to float
        img = np.float32(cv2.imread(file)) / 255.0
        facial_images.append(img)
        # For each face, detect landmarks
        facial_landmarks.append(get_landmarks(file))
        
num_images = len(facial_images)

#### Setup target image size and reference points

In [6]:
# width and height dimensions of the output image
w, h = 600, 600

# Destination eye corner positions in the output image
eye_corners_dst = [(np.int(0.3*w), np.int(h/3)), (np.int(0.7*w), np.int(h/3))]

# Add 8 boundary points for Delaunay triangulation
boundary_landmarks = np.array([(0, 0), (w/2, 0), (w-1, 0), (w-1, h/2), (w-1, h-1), (w/2, h-1), (0, h-1), (0, h/2)])

# Initializing the averaged landmarks for later computation, 68 + 8 points
averaged_landmarks = np.array([(0, 0)] * (len(facial_landmarks[0]) + len(boundary_landmarks)), np.float32)

In [7]:
# Warp images and transform landmarks to the desired output coordinate system
# and find the average of the transformed landmarks

# To store the transformed faces and points according to desired similarity transform
transformed_faces = []
transformed_points = []

for i in range(0, num_images):
    landmarks = facial_landmarks[i]
    
    # Source eye corner positions of in the orginal image
    eye_corners_src = [landmarks[36], landmarks[45]]
    
    # Get the similarity transform matrix
    sim_mat = similarity_transform(eye_corners_src, eye_corners_dst)
    
    # Apply similarity transform to the face image
    transformed_face = cv2.warpAffine(facial_images[i], sim_mat, (w, h))
    
    # Apply similarity transform on landmarks
    transformed_landmarks = cv2.transform(np.array(landmarks).reshape((68, 1, 2)), sim_mat).reshape(68, 2)
    
    # Append boundary landmarks to be used for Delaunay trigangulation
    transformed_landmarks = np.append(np.float32(transformed_landmarks), boundary_landmarks, axis=0)
    
    # Compute the averaged landmarks
    averaged_landmarks += transformed_landmarks / num_images
    
    transformed_points.append(transformed_landmarks)
    transformed_faces.append(transformed_face)

In [14]:
# If we naively average out all the faces, the final result will be a blury one
# We need to use the Delaunay triangles generated from the averaged landmarks
# as the reference to warp out each transformed faces by virtue of their Delaunay triangles
rect = (0, 0, w, h)
delaunay_triangles = calculate_delaunay_triangles(rect, averaged_landmarks)

final_output = np.zeros((h, w, 3), np.float32)

# Warp each image according to averaged image landmarks
for i in range(0, len(transformed_faces)):
    img = np.zeros((h, w, 3), np.float32)
    # Warp triangles one by one
    for j in range(0, len(delaunay_triangles)):
        src_triangle = []
        dst_triangle = []
        
        for k in range(0, 3):
            src_point = transformed_points[i][delaunay_triangles[j][k]]
            dst_point = averaged_landmarks[delaunay_triangles[j][k]]
            # Here I intentionally left out the constraint point function to see what will be different
            src_triangle.append(src_point)
            dst_triangle.append(dst_point)
        
        warp_triangle(transformed_faces[i], img, src_triangle, dst_triangle)
        
    final_output = final_output + img

final_output = final_output / num_images

cv2.imshow("Averaged faces", final_output)
cv2.waitKey(0)

Totally there are 154 triangles generated
Totally there are 142 Delaunay triangles


-1